Now that I've streamlined the MCMC process, I am going to submit multiple chains simultaneously. This notebook will make multiple, similar config files, for broad comparison. 

This may be rolled into pearce as a helper function, I haven't decided.

Update from the older SLAC notebook, which is now outdated

For rmin 0, 0.5, 1.0:

    For no ab, HSAB and CorrAB emu:

        Vpeak sham
        Mpeak sham
        HOD
        HSAB HOD 

In [31]:
import yaml
import copy
from os import path
import numpy as np
import h5py

In [32]:
emu_path = '/u/ki/swmclau2/des/'
#emu_path = '~/scratch'

In [33]:
cov_path = '/u/ki/swmclau2/Git/pearce/bin/'
# cov_path = '~/Git/pearce/bin'

In [34]:
orig_cfg_fname = path.join(cov_path, 'mcmc/nh_gg_sham_hsab_mcmc_config.yaml')
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [35]:
bsub_template="""#BSUB -q medium 
#BSUB -W 24:00
#BSUB -J {jobname}
#BSUB -oo /u/ki/swmclau2/Git/pearce/bin/mcmc/config/{jobname}.out    
#BSUB -n 8 
#BSUB -R "span[ptile=8]"

python /u/ki/swmclau2/Git/pearce/pearce/inference/initialize_mcmc.py {jobname}.yaml
python /u/ki/swmclau2/Git/pearce/pearce/inference/run_mcmc.py {jobname}.yaml
"""

In [36]:
bsub_restart_template="""#BSUB -q medium 
#BSUB -W 24:00
#BSUB -J {jobname}
#BSUB -oo /u/ki/swmclau2/Git/pearce/bin/mcmc/config/{jobname}.out    
#BSUB -n 8 
#BSUB -R "span[ptile=8]"

#python /u/ki/swmclau2/Git/pearce/pearce/inference/initialize_mcmc.py {jobname}.yaml
python /u/ki/swmclau2/Git/pearce/pearce/inference/run_mcmc.py {jobname}.yaml --restart
"""

In [37]:
import numpy as np
r_bins = np.logspace(-0.3, 2, 21)
print r_bins

[  0.50118723   0.65313055   0.85113804   1.10917482   1.44543977
   1.88364909   2.45470892   3.19889511   4.16869383   5.43250331
   7.07945784   9.22571427  12.02264435  15.6675107   20.41737945
  26.6072506   34.67368505  45.18559444  58.88436554  76.73614894
 100.        ]


In [38]:
emu_names = ['Xi', 'Knn1pCDF', 'Knn2pCDF', 'Knn4pCDF', 'Knn8pCDF']
emu_fnames = [[path.join(emu_path, 'Aemulators/xi_hsab/PearceXiHSABCosmo.hdf5')],
        [path.join(emu_path, 'Aemulators/knn_1_cdf_hsab/PearceKNN1pCDFHSABCosmo.hdf5')],
             [path.join(emu_path, 'Aemulators/knn_2_cdf_hsab/PearceKNN2pCDFHSABCosmo.hdf5')],
             [path.join(emu_path, 'Aemulators/knn_4_cdf_hsab/PearceKNN4pCDFHSABCosmo.hdf5')],
             [path.join(emu_path, 'Aemulators/knn_8_cdf_hsab/PearceKNN8pCDFHSABCosmo.hdf5')]]

emu_names.append('Xi+Knnp1CDF')
emu_fnames.append([emu_fnames[0][0], emu_fnames[1][0]])

emu_names.append('KnnpCDFAll')
emu_fnames.append([emu_fnames[1][0],emu_fnames[2][0],emu_fnames[3][0],emu_fnames[4][0]])

In [39]:
#obs_names = ['xi', 'knn_1_pcdf_cut', 'knn_2_pcdf_cut', 'knn_4_pcdf_cut', 'knn_8_pcdf_cut']
obs_names = ['xi', 'knn_1_pcdf', 'knn_2_pcdf', 'knn_4_pcdf','knn_8_pcdf']

data_fnames = [path.join(cov_path, 'mock_measurements/testbox_1_1_%s.npy'%o)
                for o in obs_names]

data_fnames.append([data_fnames[0], data_fnames[1]]) # xi X knn1
obs_names.append([obs_names[0], obs_names[1]]) # xi X knn1

data_fnames.append([data_fnames[1],data_fnames[2],data_fnames[3],data_fnames[4]]) # xi X knn1
obs_names.append([obs_names[1],obs_names[2],obs_names[3],obs_names[4]]) # xi X knn1

In [40]:
for d in data_fnames[-1]:
    print np.load(d).shape

(20,)
(20,)
(20,)
(20,)


In [41]:
15+14+11

40

In [42]:
meas_cov_fnames = []
emu_cov_fnames = []

for obs, emu, data in zip(obs_names, emu_fnames, data_fnames ):
    #if obs =='vdf':
    #    continue
    #with h5py.File(emu[0], 'r') as f:
    #    rel_meas_cov = f.attrs['rel_cov']
    #    rel_emu_cov = f.attrs['rel_emu_cov']
    
    # get rid of nans 
    #rel_meas_cov[np.isnan(rel_meas_cov)] = 1e-9
    #o = np.load(data)
    #meas_cov = rel_meas_cov*np.outer(o, o)    
    #emu_cov = rel_emu_cov*np.outer(o,o)
    print obs
    if type(obs) is str:
        meas_cov_fname = path.join(cov_path, 'covmat/%s_sim_covmat.npy'%obs)

        emu_cov_fname = path.join(cov_path, 'optimization/%s_hsab_emu_cov_lpw.npy'%obs)
    else:
        continue
        #ms = []
        #es = []
        #for o in obs:
        #    ms.append(path.join(cov_path, 'covmat/%s_sim_covmat.npy'%o))
        #    es.append(path.join(cov_path, 'optimization/%s_hsab_emu_cov_lpw.npy'%o))
            
        #emu_cov_fname = es # these are separate
    
    print meas_cov_fname, emu_cov_fname
    meas_cov_fnames.append(meas_cov_fname)
    emu_cov_fnames.append([emu_cov_fname])
    

xi
/u/ki/swmclau2/Git/pearce/bin/covmat/xi_sim_covmat.npy /u/ki/swmclau2/Git/pearce/bin/optimization/xi_hsab_emu_cov_lpw.npy
knn_1_pcdf
/u/ki/swmclau2/Git/pearce/bin/covmat/knn_1_pcdf_sim_covmat.npy /u/ki/swmclau2/Git/pearce/bin/optimization/knn_1_pcdf_hsab_emu_cov_lpw.npy
knn_2_pcdf
/u/ki/swmclau2/Git/pearce/bin/covmat/knn_2_pcdf_sim_covmat.npy /u/ki/swmclau2/Git/pearce/bin/optimization/knn_2_pcdf_hsab_emu_cov_lpw.npy
knn_4_pcdf
/u/ki/swmclau2/Git/pearce/bin/covmat/knn_4_pcdf_sim_covmat.npy /u/ki/swmclau2/Git/pearce/bin/optimization/knn_4_pcdf_hsab_emu_cov_lpw.npy
knn_8_pcdf
/u/ki/swmclau2/Git/pearce/bin/covmat/knn_8_pcdf_sim_covmat.npy /u/ki/swmclau2/Git/pearce/bin/optimization/knn_8_pcdf_hsab_emu_cov_lpw.npy
['xi', 'knn_1_pcdf']
['knn_1_pcdf', 'knn_2_pcdf', 'knn_4_pcdf', 'knn_8_pcdf']


In [43]:
# special case for xi+knn1
# TODO fix for cut case 
cov = np.load(path.join(cov_path, 'covmat/xi_knnpcdf_sim_covmat.npy'))
np.save(path.join(cov_path, 'covmat/xi_knn1pcdf_sim_covmat.npy'), cov[:35][:,:35]) #cut last 5 bins of k=1

meas_cov_fnames.append(path.join(cov_path, 'covmat/xi_knn1pcdf_sim_covmat.npy'))

emu_cov_fnames.append([emu_cov_fnames[0][0], emu_cov_fnames[1][0]])

In [44]:
# special case for knn*
#cov = np.load(path.join(cov_path, 'covmat/knn_pcdf_sim_covmat.npy'))
#np.save(path.join(cov_path, 'covmat/knn_pcdf_part_sim_covmat.npy'), cov[:60][:,:60])
#cov = np.load(path.join(cov_path, 'covmat/knn_cdf_cut_sim_covmat.npy'))
#np.save(path.join(cov_path,'covmat/knn_cdf_cut_sim_covmat2.npy' ), cov[:-9][:,:-9])
meas_cov_fnames.append(path.join(cov_path, 'covmat/knn_pcdf_cut_sim_covmat.npy'))

emu_cov_fnames.append([emu_cov_fnames[1][0],emu_cov_fnames[2][0],emu_cov_fnames[3][0],\
                       emu_cov_fnames[4][0]])

In [45]:
%matplotlib inline
from matplotlib import pyplot as plt

In [46]:
def cov_to_corr(cov):
    std = np.sqrt(np.diag(cov))
    
    denom = np.outer(std, std)
    
    return cov/denom

In [47]:
n_walkers = 250
n_steps = 110000

In [48]:
# these computed where the average values - 1 std is less than 1/sqrt(n_ran)
# ntoe these are the wins for bins not bin centers
keep_scales = {1: [True, True, True, True, True, True, True, True, True, True,\
                   True, True, True, True, True, True,True, True, True, True,\
                   True],
#2: [False, True, True, True, True, True, True, True, True, True, True, True, True,\
#    True, True,True, False, False, False, False, False],
2: [True, True, True, True, True, True, True, True, True, True, True, True, True,\
    True, True,True, True, True, True, True, True],
4: [True, True, True, True, True, True, True, True, True, True, True, True,\
    True, True, True, True, True, True, True, True, True],
#8: [False, False, False, False, False, False, False, True, True, True,\
#    True, True, True, True, True, True, True, False, False, False, False]}
8: [True, True, True, True, True, True, True, True, True, True,\
    True, True, True, True, True, True, True, True, True, True, True]}

In [49]:
k_rbins = [[list(r_bins)]]
for k in [1,2,4,8]:
    k_rbins.append([list(r_bins[keep_scales[k]])])
           
k_rbins.append([list(r_bins), list(r_bins[keep_scales[1]])])
k_rbins.append([list(r_bins[keep_scales[k]]) for k in [1,2,4,8]])

In [50]:
print obs_names

['xi', 'knn_1_pcdf', 'knn_2_pcdf', 'knn_4_pcdf', 'knn_8_pcdf', ['xi', 'knn_1_pcdf'], ['knn_1_pcdf', 'knn_2_pcdf', 'knn_4_pcdf', 'knn_8_pcdf']]


## HOD
----


In [51]:
#orig_cfg_fname = '/u/ki/swmclau2//Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [52]:
tmp_cfg = copy.deepcopy(orig_cfg)

directory = path.join(cov_path, "mcmc/config/")
output_dir = path.join(emu_path, "PearceMCMC/")

jobname_template = "HOD_1_1_rmin_{rmin}_{emu_name}"#_fixed_HOD"

for rmin in [0.1, 1.0, 5.0]:
    for emu_fname, emu_name, emu_cov, obs, meas_cov, data_name, rb in\
    zip(emu_fnames, emu_names, emu_cov_fnames, obs_names, meas_cov_fnames,\
        data_fnames, k_rbins):
        
        #if rmin is not None:
        tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin}#, 'cosmo':0}
        #else:
        #    tmp_cfg['emu']['fixed_params'] = {'z': 0.0, 'rmin':rmin, 'csomo':0}

        tmp_cfg['emu']['training_file'] = emu_fname
        tmp_cfg['emu']['emu_type'] = ['LemonPepperWet' for i in xrange(len(emu_fname))]
        #print emu_cov
        tmp_cfg['emu']['emu_cov_fname'] = emu_cov
        
        tmp_cfg['data']['obs']['obs'] = [obs] if type(obs) is not list else obs
        tmp_cfg['data']['obs']['rbins'] = rb
        tmp_cfg['data']['true_data_fname']= [data_name] if type(data_name) is not list else data_name
        tmp_cfg['data']['true_cov_fname'] = meas_cov
        
        
        tmp_cfg['data']['cov']['meas_cov_fname'] = meas_cov#tmp_cfg['data']['true_cov_fname']
        tmp_cfg['data']['cov']['emu_cov_fname'] = tmp_cfg['emu']['emu_cov_fname'] # TODO make this not be redundant
        
        jobname = jobname_template.format(rmin=rmin, emu_name=emu_name)
        tmp_cfg['fname'] = path.join(output_dir, jobname+'.hdf5')
        
        tmp_cfg['sim']= {'gal_type': 'HOD',
       'hod_name': 'zheng07',
       'hod_params': {'alpha': 1.0,
        'logM0': 13.5,
        'logM1': 14.0,
        'sigma_logM': 0.3,
        'conc_gal_bias': 0.95},
       'nd': '5e-4',
       'scale_factor': 1.0,
       'min_ptcl': 100, 
       'sim_hps': {'boxno': 1,
        'realization': 1,
        'system': 'ki-ls'},
       'simname': 'testbox'}
        
        # TODO i shouldnt have to specify this this way
        tmp_cfg['data']['sim'] = tmp_cfg['sim']
        
        tmp_cfg['chain']['nwalkers'] = n_walkers
        tmp_cfg['chain']['nsteps'] = n_steps
        tmp_cfg['chain']['mcmc_type'] = 'normal'
        
        # fix params during MCMC    
        #tmp_cfg['chain']['fixed_params'].update(tmp_cfg['sim']['hod_params'])
        
        #try:
        #    del tmp_cfg['data']['true_data_fname']
        #    del tmp_cfg['data']['true_cov_fname']
        #except KeyError:
        #    pass

        with open(path.join(directory, jobname +'.yaml'), 'w') as f:
            yaml.dump(tmp_cfg, f)

        #with open(path.join(directory, jobname + '.sbatch'), 'w') as f:
        #    f.write(sbatch_template.format(jobname=jobname))
            
        with open(path.join(directory, jobname + '.bsub'), 'w') as f:
            f.write(bsub_template.format(jobname=jobname))
            #f.write(bsub_restart_template.format(jobname=jobname))

In [53]:
from pearce.mocks.kittens import TestBox

In [54]:
cat = TestBox(1,1)
pv = cat._get_cosmo_param_names_vals()
cosmo_params = dict(zip(pv[0], pv[1]))

## HSAB
----


In [55]:
obs_names = ['xi', 'knn_1_cdf_cut', 'knn_2_cdf_cut', 'knn_4_cdf_cut', 'knn_8_cdf_cut']
#obs_names = ['xi', 'knn_1_pcdf', 'knn_2_pcdf', 'knn_4_pcdf','knn_8_pcdf']

data_fnames = [path.join(cov_path, 'mock_measurements/testbox_5_1_hsab_%s.npy'%o)
                for o in obs_names]

#data_fnames.append(copy.copy(data_fnames))

data_fnames.append([data_fnames[0], data_fnames[1]]) # xi X knn1
obs_names.append([obs_names[0], obs_names[1]]) # xi X knn1

data_fnames.append([data_fnames[1],data_fnames[2],data_fnames[3],data_fnames[4]]) # xi X knn1
obs_names.append([obs_names[1],obs_names[2],obs_names[3],obs_names[4]]) # xi X knn1

In [56]:
#orig_cfg_fname = '/u/ki/swmclau2//Git/pearce/bin/mcmc/nh_gg_sham_hsab_mcmc_config.yaml'
with open(orig_cfg_fname, 'r') as yamlfile:
    orig_cfg = yaml.load(yamlfile)

In [57]:
cat = TestBox(5,1)
pv = cat._get_cosmo_param_names_vals()
cosmo_params = dict(zip(pv[0], pv[1]))